# Part3

In [1]:
import json
from urllib.request import urlopen
import requests,json,os,sys,time,re
import pandas as pd
import numpy as np

In [14]:
def show_work_status(singleCount, totalCount, currentCount):
    currentCount+=singleCount
    percentage=1.*currentCount/totalCount*100
    status='>'*int(percentage/2)+'-'*(50-int(percentage/2))
    if percentage % 3:
        sys.stdout.write('\rStatus:[{0}]{1:.2f}%'.format(status,percentage))
        sys.stdout.flush()
    if percentage>=100: print('\n')

In [5]:
import string
content_data = pd.read_csv('Part2_pos_rate_dataframe.csv')
del content_data['Unnamed: 0']  #####     df_new = df.rename(columns={'A': 'a'}, index={'ONE': 'one'})
content_data

,Positive,Negative,Game_id,Game_name,Positive_Rate
0,196400,2471,620,Portal 2,0.987575
1,188807,3341,431960,Wallpaper Engine,0.982612
2,69831,1273,400,Portal,0.982097
3,380845,7407,292030,The Witcher 3: Wild Hunt,0.980922
4,538314,12000,105600,Terraria,0.978194
5,244377,5920,413150,Stardew Valley,0.976348
6,156967,4087,10,Counter-Strike,0.974623
7,309095,8367,227300,Euro Truck Simulator 2,0.973644
8,99915,2937,220,Half-Life 2,0.971444
9,353503,10611,550,Left 4 Dead 2,0.970858


* Add website links to each game by its game_id, but first, we need to convert game_id into str, then we can concate game_id with pre_link as string

In [12]:
content_data['Game_id'] = content_data['Game_id'].apply(str)  ########## Convert to String, Using them later  ############
content_data['links'] = ('https://store.steampowered.com/app/'+ content_data['Game_id']).astype(str)
content_data.head(5)

,Positive,Negative,Game_id,Game_name,Positive_Rate,links
0,196400,2471,620,Portal 2,0.987575,https://store.steampowered.com/app/620
1,188807,3341,431960,Wallpaper Engine,0.982612,https://store.steampowered.com/app/431960
2,69831,1273,400,Portal,0.982097,https://store.steampowered.com/app/400
3,380845,7407,292030,The Witcher 3: Wild Hunt,0.980922,https://store.steampowered.com/app/292030
4,538314,12000,105600,Terraria,0.978194,https://store.steampowered.com/app/105600


* In this step, we want to use http_requests and Beautiful_Soup as our tools to parse these games' descriptions on Steam website

In [15]:
import urllib
import bs4
from bs4 import BeautifulSoup
#url_contents = urllib.request.urlopen(url).read()
Game_description = pd.Series([]) 
current_count = 0
for i in range(100):
    #print(content_data['links'][i])
    url = content_data['links'][i]
    url_contents = urllib.request.urlopen(url).read()
    soup = bs4.BeautifulSoup(url_contents, 'html.parser')
    div = soup.find("div", {"id": "game_area_description"})
    ########## Fina the area of game descriptiona ##########
    Game_description[i]=div
    show_work_status(1,100,current_count)
    current_count+=1

Status:[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%



* Get the whole content about game_descriptions with html format

In [35]:
soup = bs4.BeautifulSoup(url_contents, 'html.parser')
div = soup.find("div", {"id": "game_area_description"})

content = str(div)
content_data['Game_description'] = Game_description
content_data['Game_description'] = content_data['Game_description'].apply(str)
content_data


,Positive,Negative,Game_id,Game_name,Positive_Rate,links,Game_description
0,196400,2471,620,Portal 2,0.987575,https://store.steampowered.com/app/620,"<div class=""game_area_description"" id=""game_ar..."
1,188807,3341,431960,Wallpaper Engine,0.982612,https://store.steampowered.com/app/431960,"<div class=""game_area_description"" id=""game_ar..."
2,69831,1273,400,Portal,0.982097,https://store.steampowered.com/app/400,"<div class=""game_area_description"" id=""game_ar..."
3,380845,7407,292030,The Witcher 3: Wild Hunt,0.980922,https://store.steampowered.com/app/292030,"<div class=""game_area_description"" id=""game_ar..."
4,538314,12000,105600,Terraria,0.978194,https://store.steampowered.com/app/105600,"<div class=""game_area_description"" id=""game_ar..."
5,244377,5920,413150,Stardew Valley,0.976348,https://store.steampowered.com/app/413150,"<div class=""game_area_description"" id=""game_ar..."
6,156967,4087,10,Counter-Strike,0.974623,https://store.steampowered.com/app/10,"<div class=""game_area_description"" id=""game_ar..."
7,309095,8367,227300,Euro Truck Simulator 2,0.973644,https://store.steampowered.com/app/227300,"<div class=""game_area_description"" id=""game_ar..."
8,99915,2937,220,Half-Life 2,0.971444,https://store.steampowered.com/app/220,"<div class=""game_area_description"" id=""game_ar..."
9,353503,10611,550,Left 4 Dead 2,0.970858,https://store.steampowered.com/app/550,"<div class=""game_area_description"" id=""game_ar..."


* Clean the HTML content with regular expression and get the text descriptions' content

In [28]:
###########################################################################################################
######################################## Clean description  ###############################################
###########################################################################################################

current_count = 0
for i in range(0,100):
    content_data['Game_description'][i] = re.sub('<.*?>', ' ', content_data['Game_description'][i])
    content_data['Game_description'][i] = re.sub('\\n.*\\n', ' ', content_data['Game_description'][i])
    content_data['Game_description'][i] = content_data['Game_description'][i].replace("\t","")
    content_data['Game_description'][i] = content_data['Game_description'][i].translate(content_data['Game_description'][i].maketrans(' ',' ',string.punctuation))

    show_work_status(1,100,current_count)
    current_count+=1
    

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Status:[--------------------------------------------------]1.00%

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Status:[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%



In [36]:
content_data.head(5)

,Positive,Negative,Game_id,Game_name,Positive_Rate,links,Game_description
0,196400,2471,620,Portal 2,0.987575,https://store.steampowered.com/app/620,"<div class=""game_area_description"" id=""game_ar..."
1,188807,3341,431960,Wallpaper Engine,0.982612,https://store.steampowered.com/app/431960,"<div class=""game_area_description"" id=""game_ar..."
2,69831,1273,400,Portal,0.982097,https://store.steampowered.com/app/400,"<div class=""game_area_description"" id=""game_ar..."
3,380845,7407,292030,The Witcher 3: Wild Hunt,0.980922,https://store.steampowered.com/app/292030,"<div class=""game_area_description"" id=""game_ar..."
4,538314,12000,105600,Terraria,0.978194,https://store.steampowered.com/app/105600,"<div class=""game_area_description"" id=""game_ar..."


* Using Tf-idf to measure the weight of every word in each game description

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel,cosine_similarity
tfidf_vectorizer = TfidfVectorizer(strip_accents = 'unicode',stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(list(content_data['Game_description']))
#Need to convert every Series in DataFrame to List format
tfidf_vectorizer.get_feature_names()
#Array mapping from feature integer indices to feature name
print(tfidf)  ####Tf-idf-weighted document-term matrix.     (0, 3473)	0.4059439800797409   (0, 1417)	0.08671197437269683

  (0, 3405)	0.40381157518152166
  (0, 1387)	0.08625648039332984
  (0, 387)	0.05978096737789393
  (0, 1856)	0.09400007154402229
  (0, 2423)	0.07301872388561188
  (0, 1947)	0.11801288524274954
  (0, 4290)	0.21251596089969305
  (0, 3004)	0.13055026546983886
  (0, 1428)	0.06144271462911911
  (0, 3167)	0.23912386951157572
  (0, 84)	0.07650070930319192
  (0, 2394)	0.07301872388561188
  (0, 114)	0.08625648039332984
  (0, 1053)	0.07007474716279817
  (0, 1107)	0.08625648039332984
  (0, 1827)	0.08076231503630434
  (0, 4087)	0.07301872388561188
  (0, 3408)	0.09400007154402229
  (0, 2462)	0.07650070930319192
  (0, 708)	0.08625648039332984
  (0, 1422)	0.05312899022492326
  (0, 3052)	0.10661390883794943
  (0, 761)	0.15007558886326805
  (0, 2273)	0.07301872388561188
  (0, 1898)	0.07301872388561188
  :	:
  (99, 259)	0.04884658245908129
  (99, 3508)	0.04884658245908129
  (99, 2709)	0.04884658245908129
  (99, 117)	0.04884658245908129
  (99, 2170)	0.04884658245908129
  (99, 4576)	0.04884658245908129
  (9

* A example of how tf-idf works in a text

In [31]:
#https://stackoverflow.com/questions/23792781/tf-idf-feature-weights-using-sklearn-feature-extraction-text-tfidfvectorizerfrom sklearn.feature_extraction.text import TfidfVectorizer
corpus = ["This is very strange strange informal",
          "This is very nice"]
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
Y = vectorizer._tfidf
idf = vectorizer.idf_
print (dict(zip(vectorizer.get_feature_names(), idf)))
print(X)  ####Tf-idf-weighted document-term matrix.
print(Y)

{'informal': 1.4054651081081644, 'is': 1.0, 'nice': 1.4054651081081644, 'strange': 1.4054651081081644, 'this': 1.0, 'very': 1.0}
  (0, 4)	0.27867523200844097
  (0, 1)	0.27867523200844097
  (0, 5)	0.27867523200844097
  (0, 3)	0.7833366301636224
  (0, 0)	0.3916683150818112
  (1, 4)	0.44832087319911734
  (1, 1)	0.44832087319911734
  (1, 5)	0.44832087319911734
  (1, 2)	0.6300993445179441
TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)


* Let's see what these descriptions like

In [32]:
list_app_name = list(content_data['Game_name'])
content_data['Game_description']    ################# Each row of pd.Series

0       Portal 2 draws from the awardwinning formula...
1       Wallpaper Engine enables you to use live wal...
2        Portal™ is a new single player game from Va...
3       The Witcher Wild Hunt is a storydriven open ...
4       Dig Fight Explore Build  The very world is a...
5       Stardew Valley is an openended countrylife R...
6       Play the worlds number 1 online action game ...
7          Travel across Europe as king of the road ...
8       1998 HALFLIFE sends a shock through the game...
9       Set in the zombie apocalypse Left 4 Dead 2 L...
10      Don’t Starve is an uncompromising wilderness...
11      Named Game of the Year by over 50 publicatio...
12      Garrys Mod is a physics sandbox There arent ...
13      Dont Starve Together is the standalone multi...
14         Episode 1 now FREE   Life is Strange is a...
15       HalfLife® 2 Episode Two is the second in a ...
16         Tomb Raider explores the intense and grit...
17           Coop roguelike action   The classic

* Using Cosine as Similarity measures,sorting Top 10 moast ralative games

In [33]:
dic_recommended_1 = {}
for index in range(content_data['Game_description'].shape[0]):
    #Iterate items in DataFrame
    print(tfidf[2])
    similarities_1 = linear_kernel(tfidf[index],tfidf).flatten()
    #print(tfidf)  ####Tf-idf-weighted document-term matrix.     (0, 3473)	0.4059439800797409   (0, 1417)	0.08671197437269683
    #print(linear_kernel(tfidf[1],tfidf[3]))
    ####For each game，tfidf value list is a very long list，tfidf[index:index+1] calculates all of the words'tfidf for one game,
    
    #similarities_2 = cosine_similarities(tfidf[index:index+1],tfidf).flatten()
    related_docs_indices_1 = (-similarities_1).argsort()[:10]
    #related_docs_indices_2 = similarities_2.argsort()[:-10:-1]

   
    #argsort returns an array of indices of the same shape as 
    dic_recommended_1.update({list_app_name[index]:[list_app_name[i] for i in related_docs_indices_1]})
df_content_based_results_1 = pd.DataFrame(dic_recommended_1)
#df_content_based_results_1.index.name = 'steam_appname'
df_content_based_results_1.reset_index(inplace=True)

  (0, 3405)	0.20311644965253686
  (0, 2423)	0.18364139199250787
  (0, 1947)	0.0989337591135353
  (0, 3052)	0.13406648036902558
  (0, 4006)	0.11909992800836852
  (0, 3360)	0.07991833560349251
  (0, 262)	0.21693422296204806
  (0, 3912)	0.20311644965253686
  (0, 1943)	0.0656409281439951
  (0, 4705)	0.08580709703882831
  (0, 3356)	0.10476289123998732
  (0, 3407)	0.21693422296204806
  (0, 4086)	0.13963066337553662
  (0, 742)	0.1545278394616429
  (0, 1950)	0.1113984457029097
  (0, 3406)	0.23640928062207706
  (0, 4778)	0.1923985520051058
  (0, 3014)	0.14294448773050655
  (0, 2587)	0.23640928062207706
  (0, 653)	0.21693422296204806
  (0, 2266)	0.23640928062207706
  (0, 3114)	0.18364139199250787
  (0, 1949)	0.17623731870004675
  (0, 2280)	0.1923985520051058
  (0, 4177)	0.21693422296204806
  (0, 3304)	0.21693422296204806
  (0, 3549)	0.20311644965253686
  (0, 3136)	0.20311644965253686
  (0, 2787)	0.23640928062207706
  (0, 3101)	0.1591057210355656
  (0, 4194)	0.17623731870004675
  (0, 3405)	0.2031

* Get the results: the most relative 10 games about game_description contents for each Top 100 popular game

In [34]:
df_content_based_results_1 = df_content_based_results_1.T
print(df_content_based_results_1.shape)
df_content_based_results_1

(101, 10)


,0,1,2,3,4,5,6,7,8,9
index,0,1,2,3,4,5,6,7,8,9
Portal 2,Portal 2,Portal,Half-Life,Call of Duty: Modern Warfare 2,Left 4 Dead 2,Infestation: The New Z,Castle Crashers,Left 4 Dead,A Story About My Uncle,Grand Theft Auto V
Wallpaper Engine,Wallpaper Engine,ARK: Survival Evolved,VRChat,Unturned,Guacamelee! Super Turbo Championship Edition,The Elder Scrolls V: Skyrim,Cities: Skylines,Euro Truck Simulator 2,Grand Theft Auto V,Dying Light
Portal,Portal,Portal 2,Half-Life 2: Episode Two,Left 4 Dead,Counter-Strike: Condition Zero,Left 4 Dead 2,Call of Duty: Modern Warfare 2,Counter-Strike: Global Offensive,Infestation: The New Z,The Tiny Bang Story
The Witcher 3: Wild Hunt,The Witcher 3: Wild Hunt,MONSTER HUNTER: WORLD,The Elder Scrolls V: Skyrim,Infestation: The New Z,Evolve Stage 2,Dying Light,Fallout 4,Black Desert Online,Tom Clancy's Rainbow Six Siege,Grim Dawn
Terraria,Terraria,Don't Starve,Infestation: The New Z,Dying Light,Trove,Chivalry: Medieval Warfare,Fallout 4,Mount & Blade II: Bannerlord,Cities: Skylines,A Story About My Uncle
Stardew Valley,Stardew Valley,Black Desert Online,Left 4 Dead 2,Don't Starve,ARK: Survival Evolved,Grim Dawn,Mount & Blade II: Bannerlord,Risk of Rain 2,MONSTER HUNTER: WORLD,Fallout Shelter
Counter-Strike,Counter-Strike,Call of Duty: Modern Warfare 2,Counter-Strike: Condition Zero,Day of Defeat,Day of Defeat: Source,Counter-Strike: Source,Rocket League,Portal 2,Chivalry: Medieval Warfare,Half-Life
Euro Truck Simulator 2,Euro Truck Simulator 2,Half-Life 2,The Elder Scrolls V: Skyrim,VRChat,Fallout 4,Grim Dawn,The Witcher 3: Wild Hunt,Dying Light,Team Fortress 2,Brawlhalla
Half-Life 2,Half-Life 2,Half-Life: Opposing Force,Half-Life,Half-Life 2: Episode One,Half-Life: Blue Shift,Counter-Strike: Global Offensive,Euro Truck Simulator 2,Black Squad,Counter-Strike: Source,Don't Starve


In [ ]:
7